# 🐳 DataLab - Gerenciamento e Monitoramento Docker

Este notebook demonstra como gerenciar e monitorar todos os serviços do ambiente DataLab usando comandos Docker. 

## 🎯 Objetivos:
- Listar e analisar containers em execução
- Verificar status de saúde dos serviços
- Inspecionar detalhes dos containers
- Gerenciar containers (parar, remover, reiniciar)
- Monitorar recursos e logs

## 🏗️ Arquitetura DataLab:
```
┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│   MinIO     │  │    Kafka    │  │    Spark    │
│  (Storage)  │  │ (Streaming) │  │(Processing) │
└─────────────┘  └─────────────┘  └─────────────┘
       │                │                │
       └────────────────┼────────────────┘
                        │
        ┌───────────────┴───────────────┐
        │                               │
┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│   MLflow    │  │   Prefect   │  │  Jupyter    │
│    (ML)     │  │(Workflows)  │  │ (Analysis)  │
└─────────────┘  └─────────────┘  └─────────────┘
        │                               │
        └───────────────┬───────────────┘
                        │
        ┌───────────────┴───────────────┐
        │                               │
┌─────────────┐              ┌─────────────┐
│ Prometheus  │              │  Streamlit  │
│(Monitoring) │              │   (Apps)    │
└─────────────┘              └─────────────┘
        │
┌─────────────┐
│   Grafana   │
│(Dashboard)  │
└─────────────┘
```

In [ ]:
# Import Required Libraries
import subprocess
import pandas as pd
import json
import time
import re
from datetime import datetime
from IPython.display import display, HTML, clear_output
import warnings

warnings.filterwarnings("ignore")

# Configuração para melhor visualização
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", 50)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📅 Notebook executado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
def run_docker_command(command, capture_output=True):
    """
    Executa um comando Docker e retorna o resultado
    """
    try:
        if capture_output:
            result = subprocess.run(command, shell=True, capture_output=True, text=True)
            if result.returncode == 0:
                return result.stdout.strip()
            else:
                return f"Erro: {result.stderr.strip()}"
        else:
            result = subprocess.run(command, shell=True, text=True)
            return "Comando executado"
    except Exception as e:
        return f"Erro ao executar comando: {str(e)}"


def format_size(size_str):
    """
    Formata o tamanho para melhor legibilidade
    """
    if not size_str or size_str == "N/A":
        return "N/A"

    # Converte diferentes formatos de tamanho
    size_str = size_str.upper()
    if "B" in size_str:
        return size_str
    else:
        return f"{size_str}B"


def parse_docker_ps_output(output):
    """
    Converte a saída do docker ps em um DataFrame
    """
    lines = output.strip().split("\n")
    if len(lines) < 2:
        return pd.DataFrame()

    headers = [
        "CONTAINER_ID",
        "IMAGE",
        "COMMAND",
        "CREATED",
        "STATUS",
        "PORTS",
        "NAMES",
    ]

    containers = []
    for line in lines[1:]:  # Pula o cabeçalho
        # Usa regex para dividir corretamente considerando espaços
        parts = re.split(r"\s{2,}", line.strip())
        if len(parts) >= 7:
            containers.append(parts[:7])
        elif len(parts) >= 6:
            containers.append(parts + [""])  # Adiciona campo vazio se necessário

    return pd.DataFrame(containers, columns=headers)


print("🔧 Funções utilitárias definidas com sucesso!")

## 📋 1. Listar Containers em Execução

Vamos examinar todos os containers DataLab que estão rodando atualmente:

In [ ]:
# Listar todos os containers em execução
print("🔍 Executando: docker ps")
print("=" * 50)

docker_ps_output = run_docker_command("docker ps")
print(docker_ps_output)

print("\n" + "=" * 50)
print(f"⏰ Comando executado em: {datetime.now().strftime('%H:%M:%S')}")

In [ ]:
# Converter saída em DataFrame para melhor análise
df_containers = parse_docker_ps_output(docker_ps_output)

if not df_containers.empty:
    print("📊 Containers DataLab em execução:")
    print("=" * 50)

    # Mostrar informações resumidas
    display(df_containers[["NAMES", "IMAGE", "STATUS", "PORTS"]].head(10))

    print(f"\n📈 Total de containers em execução: {len(df_containers)}")

    # Estatísticas por imagem
    image_counts = df_containers["IMAGE"].value_counts()
    print("\n🐳 Distribuição por imagem:")
    for image, count in image_counts.items():
        print(f"  • {image}: {count} container(s)")
else:
    print("⚠️ Nenhum container encontrado em execução")

## 🔍 2. Filtrar Containers por Status de Saúde

Vamos analisar o status de saúde de cada serviço do DataLab:

In [ ]:
# Mapear serviços do DataLab
datalab_services = {
    "minio": "🗄️ MinIO (Object Storage)",
    "kafka": "🔄 Kafka (Message Streaming)",
    "spark-master": "⚡ Spark Master (Processing)",
    "spark-worker": "🔧 Spark Worker (Processing)",
    "mlflow": "🤖 MLflow (ML Lifecycle)",
    "prefect": "🔀 Prefect (Workflow Orchestration)",
    "jupyterhub": "📓 JupyterHub (Notebooks)",
    "streamlit": "🖥️ Streamlit (Web Apps)",
    "prometheus": "📊 Prometheus (Metrics)",
    "grafana": "📈 Grafana (Dashboards)",
}

print("🏥 Status de Saúde dos Serviços DataLab")
print("=" * 60)

healthy_services = []
unhealthy_services = []
running_services = []

for _, container in df_containers.iterrows():
    name = container["NAMES"]
    status = container["STATUS"]

    # Extrair nome base do serviço
    base_name = name.split("-")[0] if "-" in name else name

    if base_name in datalab_services:
        service_description = datalab_services[base_name]

        if "healthy" in status.lower():
            healthy_services.append((name, service_description))
            status_icon = "✅"
        elif "unhealthy" in status.lower():
            unhealthy_services.append((name, service_description))
            status_icon = "❌"
        else:
            running_services.append((name, service_description))
            status_icon = "🟡"

        print(f"{status_icon} {service_description}")
        print(f"   Container: {name}")
        print(f"   Status: {status}")
        print()

print("📊 Resumo do Status:")
print(f"✅ Serviços saudáveis: {len(healthy_services)}")
print(f"❌ Serviços com problemas: {len(unhealthy_services)}")
print(f"🟡 Serviços rodando (status desconhecido): {len(running_services)}")

if unhealthy_services:
    print("\n⚠️ Serviços que precisam de atenção:")
    for name, desc in unhealthy_services:
        print(f"  • {desc} ({name})")

In [ ]:
# Verificar conectividade dos principais serviços
import requests
from urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

print("🌐 Teste de Conectividade dos Serviços")
print("=" * 50)

service_urls = {
    "MinIO Console": "http://localhost:9001",
    "Spark Master UI": "http://localhost:8080",
    "MLflow UI": "http://localhost:5000",
    "Prefect UI": "http://localhost:4200",
    "JupyterHub": "http://localhost:8888",
    "Streamlit": "http://localhost:8501",
    "Prometheus": "http://localhost:9090",
    "Grafana": "http://localhost:3000",
}

connectivity_results = {}

for service, url in service_urls.items():
    try:
        response = requests.get(url, timeout=5, verify=False)
        if response.status_code == 200:
            connectivity_results[service] = "✅ Acessível"
            status = "✅"
        else:
            connectivity_results[service] = f"⚠️ HTTP {response.status_code}"
            status = "⚠️"
    except requests.exceptions.RequestException as e:
        connectivity_results[service] = "❌ Não acessível"
        status = "❌"

    print(f"{status} {service:20} - {url}")

print("\n📊 Resumo de Conectividade:")
accessible = sum(1 for result in connectivity_results.values() if "✅" in result)
total = len(connectivity_results)
print(f"Serviços acessíveis: {accessible}/{total} ({accessible/total*100:.1f}%)")

## 🔍 3. Inspecionar Detalhes dos Containers

Vamos examinar informações detalhadas de containers específicos:

In [ ]:
def inspect_container(container_name):
    """
    Inspeciona um container específico e retorna informações detalhadas
    """
    print(f"🔍 Inspecionando container: {container_name}")
    print("=" * 50)

    # Obter informações básicas
    inspect_cmd = f"docker inspect {container_name}"
    inspect_output = run_docker_command(inspect_cmd)

    try:
        inspect_data = json.loads(inspect_output)[0]

        # Informações básicas
        print("📋 Informações Básicas:")
        print(f"  • ID: {inspect_data['Id'][:12]}")
        print(f"  • Nome: {inspect_data['Name'][1:]}")
        print(f"  • Imagem: {inspect_data['Config']['Image']}")
        print(f"  • Estado: {inspect_data['State']['Status']}")
        print(f"  • Criado em: {inspect_data['Created'][:19]}")
        print(f"  • Iniciado em: {inspect_data['State'].get('StartedAt', 'N/A')[:19]}")

        # Configuração de rede
        print("\n🌐 Configuração de Rede:")
        networks = inspect_data["NetworkSettings"]["Networks"]
        for net_name, net_info in networks.items():
            print(f"  • Rede: {net_name}")
            print(f"    - IP: {net_info.get('IPAddress', 'N/A')}")
            print(f"    - Gateway: {net_info.get('Gateway', 'N/A')}")

        # Mapeamento de portas
        print("\n🔌 Mapeamento de Portas:")
        port_bindings = inspect_data["NetworkSettings"].get("Ports", {})
        if port_bindings:
            for container_port, host_bindings in port_bindings.items():
                if host_bindings:
                    for binding in host_bindings:
                        print(
                            f"  • {binding['HostPort']}:{container_port.split('/')[0]}"
                        )
                else:
                    print(f"  • {container_port} (não mapeada)")
        else:
            print("  Nenhuma porta mapeada")

        # Volumes
        print("\n💾 Volumes Montados:")
        mounts = inspect_data.get("Mounts", [])
        if mounts:
            for mount in mounts:
                print(f"  • {mount['Source']} → {mount['Destination']}")
                print(f"    Tipo: {mount['Type']}, Modo: {mount.get('Mode', 'N/A')}")
        else:
            print("  Nenhum volume montado")

        # Recursos
        print("\n⚡ Configuração de Recursos:")
        resources = inspect_data["HostConfig"]
        print(f"  • Memória: {resources.get('Memory', 0) or 'Ilimitada'}")
        print(f"  • CPU Shares: {resources.get('CpuShares', 0) or 'Padrão'}")
        print(
            f"  • Restart Policy: {resources.get('RestartPolicy', {}).get('Name', 'N/A')}"
        )

        # Health check se disponível
        if "Health" in inspect_data["State"]:
            health = inspect_data["State"]["Health"]
            print("\n🏥 Status de Saúde:")
            print(f"  • Status: {health['Status']}")
            print(f"  • Verificações falharam: {health['FailingStreak']}")
            if health["Log"]:
                last_check = health["Log"][-1]
                print(f"  • Última verificação: {last_check['Start'][:19]}")
                print(f"  • Código de saída: {last_check['ExitCode']}")

    except (json.JSONDecodeError, KeyError, IndexError) as e:
        print(f"❌ Erro ao processar informações do container: {e}")
        print(f"Saída bruta: {inspect_output[:200]}...")


# Exemplo: inspecionar o container do MinIO
if not df_containers.empty:
    minio_containers = df_containers[
        df_containers["NAMES"].str.contains("minio", case=False)
    ]
    if not minio_containers.empty:
        container_name = minio_containers.iloc[0]["NAMES"]
        inspect_container(container_name)
    else:
        print("⚠️ Container MinIO não encontrado")
else:
    print("⚠️ Nenhum container em execução para inspecionar")

In [ ]:
# Monitorar uso de recursos dos containers
print("\n📊 Uso de Recursos dos Containers")
print("=" * 50)

# Obter estatísticas de uso
stats_cmd = "docker stats --no-stream --format 'table {{.Container}}\\t{{.CPUPerc}}\\t{{.MemUsage}}\\t{{.MemPerc}}\\t{{.NetIO}}\\t{{.BlockIO}}'"
stats_output = run_docker_command(stats_cmd)

print("💻 Estatísticas de Recursos (snapshot atual):")
print(stats_output)

# Análise adicional de espaço em disco
print("\n💾 Uso de Espaço em Disco:")
print("-" * 30)

# Tamanho dos containers
size_cmd = "docker ps --size --format 'table {{.Names}}\\t{{.Size}}'"
size_output = run_docker_command(size_cmd)
print("📦 Tamanho dos Containers:")
print(size_output)

# Informações sobre volumes
print("\n📂 Volumes Docker:")
volumes_cmd = "docker volume ls"
volumes_output = run_docker_command(volumes_cmd)
print(volumes_output)

# Espaço total usado pelo Docker
print("\n🗄️ Uso Total de Espaço Docker:")
system_df_cmd = "docker system df"
system_df_output = run_docker_command(system_df_cmd)
print(system_df_output)

## ⚙️ 4. Gerenciamento de Containers

**⚠️ ATENÇÃO:** As operações abaixo podem afetar o funcionamento do ambiente DataLab. Use com cuidado!

In [ ]:
def show_container_logs(container_name, tail_lines=50):
    """
    Mostra os logs de um container específico
    """
    print(f"📄 Logs do container {container_name} (últimas {tail_lines} linhas):")
    print("=" * 60)

    logs_cmd = f"docker logs --tail {tail_lines} {container_name}"
    logs_output = run_docker_command(logs_cmd)

    if "Erro:" not in logs_output:
        # Truncar logs muito longos
        lines = logs_output.split("\n")
        if len(lines) > tail_lines:
            print(f"... (mostrando últimas {tail_lines} linhas de {len(lines)} total)")
            logs_output = "\n".join(lines[-tail_lines:])

        print(logs_output)
    else:
        print(f"❌ {logs_output}")


def restart_container(container_name, confirm=True):
    """
    Reinicia um container específico
    """
    if confirm:
        print(
            f"⚠️ Você tem certeza que deseja reiniciar o container '{container_name}'?"
        )
        print("Digite 'SIM' para confirmar:")
        # Em um ambiente interativo, você usaria input()
        # confirmation = input().strip()
        # if confirmation != 'SIM':
        #     print("❌ Operação cancelada")
        #     return
        print("🔄 Para executar, descomente e use input() para confirmação")
        return

    print(f"🔄 Reiniciando container {container_name}...")
    restart_cmd = f"docker restart {container_name}"
    result = run_docker_command(restart_cmd)

    if "Erro:" not in result:
        print(f"✅ Container {container_name} reiniciado com sucesso")
    else:
        print(f"❌ Erro ao reiniciar: {result}")


def stop_container(container_name, confirm=True):
    """
    Para um container específico
    """
    if confirm:
        print(f"⚠️ Você tem certeza que deseja parar o container '{container_name}'?")
        print("Digite 'SIM' para confirmar:")
        print("🛑 Para executar, descomente e use input() para confirmação")
        return

    print(f"🛑 Parando container {container_name}...")
    stop_cmd = f"docker stop {container_name}"
    result = run_docker_command(stop_cmd)

    if "Erro:" not in result:
        print(f"✅ Container {container_name} parado com sucesso")
    else:
        print(f"❌ Erro ao parar: {result}")


print("🔧 Funções de gerenciamento definidas:")
print("  • show_container_logs(container_name, tail_lines=50)")
print("  • restart_container(container_name, confirm=True)")
print("  • stop_container(container_name, confirm=True)")
print("\n💡 Exemplo de uso: show_container_logs('minio', 20)")

In [ ]:
# Exemplo: Visualizar logs do serviço com problemas (se houver)
if unhealthy_services:
    print("🔍 Analisando logs dos serviços com problemas:")
    print("=" * 50)

    for container_name, service_desc in unhealthy_services[:1]:  # Apenas o primeiro
        print(f"\n📄 Logs do {service_desc}:")
        show_container_logs(container_name, 30)
else:
    # Se todos estão saudáveis, mostrar logs do Prefect (que estava unhealthy)
    prefect_containers = df_containers[
        df_containers["NAMES"].str.contains("prefect", case=False)
    ]
    if not prefect_containers.empty:
        container_name = prefect_containers.iloc[0]["NAMES"]
        print(f"📄 Logs do Prefect (para diagnóstico):")
        show_container_logs(container_name, 30)

# Mostrar comandos úteis para troubleshooting
print("\n🛠️ Comandos Úteis para Troubleshooting:")
print("=" * 40)
print("# Ver logs em tempo real:")
print("docker logs -f <container_name>")
print("\n# Executar comando dentro do container:")
print("docker exec -it <container_name> /bin/bash")
print("\n# Verificar processos no container:")
print("docker exec <container_name> ps aux")
print("\n# Ver configuração de rede:")
print("docker network ls")
print("docker network inspect datalab_dataflow-network")

## 📊 5. Monitoramento Contínuo

Funções para monitoramento contínuo do ambiente DataLab:

In [ ]:
def monitor_datalab_health(duration_minutes=5, check_interval=30):
    """
    Monitora a saúde do ambiente DataLab continuamente
    """
    print(
        f"🔄 Iniciando monitoramento por {duration_minutes} minutos (verificação a cada {check_interval}s)"
    )
    print("=" * 70)

    end_time = time.time() + (duration_minutes * 60)
    check_count = 0

    try:
        while time.time() < end_time:
            check_count += 1
            current_time = datetime.now().strftime("%H:%M:%S")

            print(f"\n⏰ Verificação #{check_count} - {current_time}")
            print("-" * 40)

            # Verificar containers
            ps_output = run_docker_command(
                "docker ps --format 'table {{.Names}}\\t{{.Status}}'"
            )
            lines = ps_output.split("\n")

            healthy_count = 0
            unhealthy_count = 0
            running_count = 0

            for line in lines[1:]:  # Skip header
                if line.strip():
                    parts = line.split("\t")
                    if len(parts) >= 2:
                        name, status = parts[0], parts[1]
                        if any(
                            service in name.lower()
                            for service in datalab_services.keys()
                        ):
                            if "healthy" in status.lower():
                                healthy_count += 1
                            elif "unhealthy" in status.lower():
                                unhealthy_count += 1
                                print(f"❌ {name}: {status}")
                            else:
                                running_count += 1

            total_services = healthy_count + unhealthy_count + running_count
            health_percentage = (
                (healthy_count / total_services * 100) if total_services > 0 else 0
            )

            print(
                f"📊 Status: {healthy_count}✅ {unhealthy_count}❌ {running_count}🟡 ({health_percentage:.1f}% saudável)"
            )

            # Verificar uso de recursos básico
            if check_count % 3 == 0:  # A cada 3 verificações
                stats_output = run_docker_command(
                    "docker stats --no-stream --format '{{.Container}}: CPU {{.CPUPerc}}, RAM {{.MemPerc}}'"
                )
                if stats_output and "Erro:" not in stats_output:
                    print("💻 Recursos (top 3):")
                    for line in stats_output.split("\n")[:3]:
                        if line.strip():
                            print(f"  {line}")

            if time.time() < end_time:  # Não esperar na última iteração
                time.sleep(check_interval)

    except KeyboardInterrupt:
        print("\n⏹️ Monitoramento interrompido pelo usuário")

    print(f"\n✅ Monitoramento concluído após {check_count} verificações")


def generate_health_report():
    """
    Gera um relatório completo de saúde do ambiente
    """
    print("📋 RELATÓRIO DE SAÚDE DO DATALAB")
    print("=" * 50)
    print(f"📅 Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # Status dos containers
    ps_output = run_docker_command("docker ps")
    df_current = parse_docker_ps_output(ps_output)

    print(f"\n🐳 CONTAINERS ({len(df_current)} em execução):")
    for _, container in df_current.iterrows():
        name = container["NAMES"]
        status = container["STATUS"]

        if "healthy" in status.lower():
            icon = "✅"
        elif "unhealthy" in status.lower():
            icon = "❌"
        else:
            icon = "🟡"

        print(f"  {icon} {name}: {status}")

    # Conectividade
    print(f"\n🌐 CONECTIVIDADE:")
    accessible_count = 0
    for service, url in service_urls.items():
        try:
            response = requests.get(url, timeout=3, verify=False)
            if response.status_code == 200:
                print(f"  ✅ {service}: Acessível")
                accessible_count += 1
            else:
                print(f"  ⚠️ {service}: HTTP {response.status_code}")
        except:
            print(f"  ❌ {service}: Não acessível")

    connectivity_rate = (accessible_count / len(service_urls)) * 100
    print(f"\n📊 Taxa de conectividade: {connectivity_rate:.1f}%")

    # Recursos
    print(f"\n💻 RECURSOS:")
    system_df = run_docker_command("docker system df")
    if system_df and "Erro:" not in system_df:
        print(system_df)

    # Recomendações
    print(f"\n💡 RECOMENDAÇÕES:")
    if accessible_count < len(service_urls):
        print("  • Verificar serviços não acessíveis")
    if len(df_current) < 8:  # Esperamos pelo menos 8 serviços principais
        print("  • Alguns serviços podem não estar rodando")
    print("  • Executar limpeza periódica: docker system prune")
    print("  • Monitorar logs dos serviços críticos")

    return {
        "containers_running": len(df_current),
        "connectivity_rate": connectivity_rate,
        "timestamp": datetime.now().isoformat(),
    }


print("📊 Funções de monitoramento definidas:")
print("  • monitor_datalab_health(duration_minutes=5, check_interval=30)")
print("  • generate_health_report()")
print("\n💡 Execute generate_health_report() para um relatório completo")

In [ ]:
# Gerar relatório de saúde atual
health_report = generate_health_report()

print(f"\n📈 Resumo Executivo:")
print(f"  • Containers ativos: {health_report['containers_running']}")
print(f"  • Taxa de conectividade: {health_report['connectivity_rate']:.1f}%")
print(f"  • Timestamp: {health_report['timestamp']}")

## 🎯 6. Comandos Úteis e Próximos Passos

### 🔧 Comandos Docker Essenciais:

```bash
# Gerenciamento básico
docker ps                          # Listar containers
docker ps -a                       # Incluir containers parados
docker images                      # Listar imagens
docker system df                   # Uso de espaço

# Logs e debugging
docker logs -f <container>         # Logs em tempo real
docker exec -it <container> bash   # Acessar container
docker inspect <container>         # Detalhes do container

# Limpeza
docker system prune                # Limpar recursos não utilizados
docker volume prune               # Limpar volumes órfãos
docker container prune            # Remover containers parados

# Docker Compose (DataLab)
cd /home/magnomatos/Documentos/projetos-pessoais/datalab
./manage.sh status                # Status dos serviços
./manage.sh health               # Verificação de saúde
./manage.sh logs <service>       # Logs específicos
./manage.sh restart <service>    # Reiniciar serviço
```

### 🌐 URLs dos Serviços:
- **MinIO Console**: http://localhost:9001 (admin/admin123)
- **Spark Master**: http://localhost:8080
- **MLflow**: http://localhost:5000
- **Prefect**: http://localhost:4200
- **JupyterHub**: http://localhost:8888
- **Streamlit**: http://localhost:8501
- **Prometheus**: http://localhost:9090
- **Grafana**: http://localhost:3000 (admin/admin)

### 📋 Checklist de Manutenção:
- [ ] Verificar logs de serviços com problemas
- [ ] Monitorar uso de recursos (CPU, RAM, Disco)
- [ ] Fazer backup dos volumes importantes
- [ ] Atualizar imagens Docker periodicamente
- [ ] Configurar alertas no Grafana
- [ ] Documentar procedimentos de recovery

### 🚨 Troubleshooting:
1. **Serviço não responde**: Verificar logs e reiniciar
2. **Alto uso de recursos**: Ajustar limites no docker-compose
3. **Problemas de rede**: Verificar conectividade entre containers
4. **Espaço em disco**: Executar limpeza do Docker
5. **Containers não iniciam**: Verificar dependências e volumes

In [ ]:
# 🎉 Finalização do Notebook
print("🎉 NOTEBOOK DE GERENCIAMENTO DOCKER CONCLUÍDO")
print("=" * 60)
print(f"📅 Executado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(
    f"🐳 Containers analisados: {len(df_containers) if not df_containers.empty else 0}"
)
print(f"🌐 Serviços verificados: {len(service_urls)}")

print("\n✅ Principais funcionalidades disponíveis:")
print("  • Listagem e análise de containers")
print("  • Verificação de status de saúde")
print("  • Inspeção detalhada de containers")
print("  • Monitoramento de recursos")
print("  • Geração de relatórios de saúde")
print("  • Funções de gerenciamento (logs, restart, stop)")

print("\n💡 Para monitoramento contínuo, execute:")
print("monitor_datalab_health(duration_minutes=2, check_interval=15)")

print("\n🔗 Links úteis:")
for service, url in service_urls.items():
    print(f"  • {service}: {url}")

print("\n🚀 Ambiente DataLab pronto para uso!")